# Warning
根据类别关键词反向映射数据集，实现快速标注。<br>
该功能仅测试，可能导致模型效果不稳定，实际模型开发过程慎用！

In [35]:
import pandas as pd
from Utils import *

In [17]:
df_origin = pd.read_csv("../datasets/twcs.csv")[["tweet_id", "text"]]
df_origin.head(10)

,tweet_id,text
0,1,@115712 I understand. I would like to assist y...
1,2,@sprintcare and how do you propose we do that
2,3,@sprintcare I have sent several private messag...
3,4,@115712 Please send us a Private Message so th...
4,5,@sprintcare I did.
5,6,@115712 Can you please send us a private messa...
6,8,@sprintcare is the worst customer service
7,11,@115713 This is saddening to hear. Please shoo...
8,12,@sprintcare You gonna magically change your co...
9,15,@115713 We understand your concerns and we'd l...


In [18]:
# Count Origin
# 2811774
df_origin.count()

tweet_id    2811774
text        2811774
dtype: int64

# 一级逆向映射标签，通用风险词

In [20]:
keywords_general = [
    'refund', 'crash', 'error', 
    'late', 'scam', 'rude', 'login', 'worst',
]

pattern = '|'.join(keywords_general)
df_level_1__suspect = df_origin[df_origin['text'].str.contains(pattern, case=False, na=False)].set_index('tweet_id')

df_level_1__suspect.count()

text    156389
dtype: int64

In [21]:
# 156389
df_level_1__suspect.head(100)

,text
tweet_id,
8,@sprintcare is the worst customer service
36,somebody from @VerizonSupport please help meee...
49,@115722 tried to pay a bill for 60 days. No se...
59,@115722 is the worst ISP I’ve ever had
161,"@ChipotleTweets Tried, didn't work. How rude :/"
...,...
2577,@115830 amazonuk took money without any https:...
2645,@115858 when will the bug in the calendar app ...
2666,I keep getting this error message saying “the ...


In [22]:
df_level_1__suspect.to_csv("../datasets/suspect_level_1.csv")

# 二级逆向映射标签，分类词
 

In [24]:
# ✅ Enum -> dict（正向映射）
# label_to_priority = {label.name.lower(): label.value for label in Label}

# 🔁 dict -> Enum（反向映射）
# priority_to_label = {label.value: label.name.lower() for label in Label}

In [39]:
df_suspect_L1 = pd.read_csv("../assets/suspect_level_1.csv")
df_suspect_L1.head(10)

,tweet_id,text
0,8,@sprintcare is the worst customer service
1,36,somebody from @VerizonSupport please help meee...
2,49,@115722 tried to pay a bill for 60 days. No se...
3,59,@115722 is the worst ISP I’ve ever had
4,161,"@ChipotleTweets Tried, didn't work. How rude :/"
5,212,@ATVIAssist @115755 I'm trying to buy the digi...
6,214,@115756 Hello there. Sorry for the late respo...
7,267,@115768 Jeffrey I am sorry to hear that. If yo...
8,278,"@115772 Hi there, Lightroom 7.0.1 update was l..."
9,308,@115783 Misrepresentation by host. Asked them ...


In [40]:
df_L2_technical = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_technical,
    RiskLabel.TECHNICAL.name.lower(),
).set_index('tweet_id')

df_L2_technical.head(10)

,text,label
tweet_id,,
212,@ATVIAssist @115755 I'm trying to buy the digi...,technical
267,@115768 Jeffrey I am sorry to hear that. If yo...,technical
323,@1520 @XboxSupport the 5 app on Xboxes not wo...,technical
600,@115811 Hi - are you receiving an error messag...,technical
1161,@TMobileHelp trying to redeem a free tuesday c...,technical
1217,@115929 Please DM us a picture of the error yo...,technical
1219,@SouthwestAir consider this your 1st reported ...,technical
1221,"@115929 Hm, we haven't had any reported errors...",technical
1222,"@SouthwestAir yup all 400 errors, even turned ...",technical


In [42]:
# 47410
df_L2_technical.count()

text     47410
label    47410
dtype: int64

In [43]:
df_L2_technical.to_csv("../assets/labeled_L2__1_technical.csv", )

In [54]:
df_L2_payment = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_payment, 
    RiskLabel.PAYMENT.name.lower()
).set_index('tweet_id')

df_L2_payment.head(10)

,text,label
tweet_id,,
308,@115783 Misrepresentation by host. Asked them ...,payment
371,@115802 @AirAsiaSupport Why are you not refund...,payment
658,@115839 I'm sorry your book arrived this way! ...,payment
664,"@AmazonHelp delivery I paid for today,didn’t a...",payment
669,@AmazonHelp I just said it's been sorted! I've...,payment
1207,@VirginTrains For at least the third time this...,payment
1225,@AskeBay sure will do. Getting refunded by the...,payment
1531,@116025 Did you view our link? We can give a r...,payment
1532,@AskeBay I would appreciate that I got the ref...,payment


In [55]:
# 39851
df_L2_payment.count()

text     39851
label    39851
dtype: int64

In [56]:
df_L2_payment.to_csv("../assets/labeled_L2__2_payment.csv", )

In [57]:
df_L2_delivery = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_delivery, 
    RiskLabel.DELIVERY.name.lower()
).set_index('tweet_id')

df_L2_delivery.head(10)

,text,label
tweet_id,,
214,@115756 Hello there. Sorry for the late respo...,delivery
278,"@115772 Hi there, Lightroom 7.0.1 update was l...",delivery
599,@115810 Chocolate chip cookies + McFlurry = ev...,delivery
634,@115821 @AmazonHelp why is my order at my loca...,delivery
655,@115838 Provide your details here: https://t.c...,delivery
664,"@AmazonHelp delivery I paid for today,didn’t a...",delivery
686,@AmazonHelp Already contacted 3-4 times in the...,delivery
1203,@VirginTrains Always some supposedly acceptabl...,delivery
1205,@VirginTrains The 06:20 from Euston. Now I'm h...,delivery


In [58]:
# 60139
df_L2_delivery.count()

text     60139
label    60139
dtype: int64

In [59]:
df_L2_delivery.to_csv("../assets/labeled_L2__3_delivery.csv", )

In [60]:
df_L2_account = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_account, 
    RiskLabel.ACCOUNT.name.lower()
).set_index('tweet_id')

df_L2_account.head(10)

,text,label
tweet_id,,
2257,@AskPlayStation Hi! Having issue with an old a...,account
2717,"@Uber_Support I need help with my Account, It ...",account
3484,"@116535 Hi again, do you have two emails, a wo...",account
3733,"@AmazonHelp um, my account was locked as soon ...",account
5426,"@hulu_support the Hulu app logged me out, &amp...",account
6558,@117157 Why is your website always down? Pret...,account
6615,"@117167 Hey Terry, that doesn't sound good! Di...",account
6655,@117176 Having issues getting the website to l...,account
7833,@ATVIAssist @117333 the www-less domain of the...,account


In [61]:
# 6060
df_L2_account.count()

text     6060
label    6060
dtype: int64

In [62]:
df_L2_account.to_csv("../assets/labeled_L2__4_account.csv", )

In [63]:
df_L2_service = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_service, 
    RiskLabel.SERVICE.name.lower()
).set_index('tweet_id')

df_L2_service.head(10)

,text,label
tweet_id,,
8,@sprintcare is the worst customer service,service
36,somebody from @VerizonSupport please help meee...,service
49,@115722 tried to pay a bill for 60 days. No se...,service
161,"@ChipotleTweets Tried, didn't work. How rude :/",service
308,@115783 Misrepresentation by host. Asked them ...,service
323,@1520 @XboxSupport the 5 app on Xboxes not wo...,service
371,@115802 @AirAsiaSupport Why are you not refund...,service
1177,@VirginTrains so i wait almost 3 hours and the...,service
1480,@HPSupport [Laserjet Pro M102w will not print....,service


In [64]:
# 32386
df_L2_service.count()

text     32386
label    32386
dtype: int64

In [65]:
df_L2_service.to_csv("../assets/labeled_L2__5_service.csv", )

In [69]:
df_L2_pricing = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_pricing, 
    RiskLabel.PRICING.name.lower()
).set_index('tweet_id')

df_L2_pricing.head(10)

,text,label
tweet_id,,
1718,"I ordered some parts of my costume late, but m...",pricing
1860,@Delta Can I get the 35$ refunded for the seat...,pricing
2719,@Uber_Support @115877 you haven’t “reached out...,pricing
3246,"@Delta :( Spent money on a ticket, online syst...",pricing
3262,@Delta I went to the service center &amp; spok...,pricing
5511,@ChipotleTweets Went to get my boorito &amp; w...,pricing
5735,#upsscam why ups always charge cod ? Why they ...,pricing
5767,@NeweggService Purchased a case yesterday that...,pricing
5963,@AirAsiaSupport Fairness is NOT price gouging ...,pricing


In [70]:
df_L2_pricing.count()

text     2687
label    2687
dtype: int64

In [71]:
df_L2_pricing.to_csv("../assets/labeled_L2__6_pricing.csv", )

In [66]:
df_L2_legal = auto_label_by_keywords(
    df_suspect_L1, 
    'text', 
    keywords_legal, 
    RiskLabel.LEGAL.name.lower()
).set_index('tweet_id')

df_L2_legal.head(10)

,text,label
tweet_id,,
308,@115783 Misrepresentation by host. Asked them ...,legal
1929,@AmericanAir i don't think it changes within m...,legal
2299,@116231 Hello! It's our utmost pleasure to hel...,legal
2302,@116231 Let us guide you to report scams. On t...,legal
2305,@MicrosoftHelps I apologize for sounding like ...,legal
5735,#upsscam why ups always charge cod ? Why they ...,legal
9756,Never use asurion mobile insurance again!! Com...,legal
9780,@AmazonHelp I've returned several things latel...,legal
9955,@AmazonHelp amazon just doesn’t seem to get it...,legal


In [67]:
# 5499
df_L2_legal.count()

text     5499
label    5499
dtype: int64

In [68]:
df_L2_legal.to_csv("../assets/labeled_L2__7_legal.csv", )